In [1]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/bgmerin1/src/Repositorios/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))

    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

Added '/var/SP/data/home/bgmerin1/src/Repositorios/use-cases' to path
Added '/var/SP/data/home/bgmerin1/src/Repositorios' to path


In [2]:
from pyspark.sql.functions import greatest, lower, upper, trim,least
from pyspark.sql.functions import collect_set, col, lpad, lit, collect_list, desc, asc, mean as sql_mean, sum as sql_sum, datediff, count as sql_count
#from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract, datediff, to_date, from_unixtime, unix_timestamp, desc, when, col, lit, udf, size, \
    array, isnan, upper, coalesce, length, lower, concat, create_map, sum as sql_sum, greatest, max as sql_max, sort_array
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
import re
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql.functions import datediff
import numpy as np
from pyspark.sql.types import FloatType
from pyspark.sql.functions import lag, mean as sql_avg
import os
import sys
import time
from pyspark.sql.functions import count

In [3]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

In [4]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.medium"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS
    

In [5]:
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

In [6]:
def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [7]:
spark = initialize("Analisys SLA",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 35.6452200413 secs | default parallelism=2


In [8]:
customer_master=spark.read.parquet('/data/attributes/vf_es/trigger_analysis/customer_master/year=2019/month=5/day=21')


In [9]:
customer_master_ppF=customer_master.filter(col('segment_nif')!='Pure_prepaid')
customer_master_ppF.count()

3899811

In [10]:
ccc=spark.read.parquet('/data/attributes/vf_es/trigger_analysis/ccc/year=2019/month=5/day=21')
ccc.count()

5540911

In [11]:
ccc_ppF=customer_master_ppF.join(ccc,'NIF_CLIENTE','left')
ccc_ppF.count()

3899811

In [12]:
cccFiltered=ccc_ppF.select(['nif_cliente','CHURN_CANCELLATIONS_w4','label']).filter(col('CHURN_CANCELLATIONS_w4')==0)
cccFiltered.count()

3795363

In [13]:
dfSLA=spark.read.parquet('/user/bgmerin1/SLA/year=2019/month=5/day=21')

In [14]:
dfSLALabeled=customer_master.join(dfSLA,['nif_cliente'],'inner')
dfSLALabeled.count()

2292929

In [15]:
dfSLALabeled.groupBy('label').count().show()

+-----+-------+
|label|  count|
+-----+-------+
|  0.0|2211383|
|  1.0|  81546|
+-----+-------+



In [16]:
dfSLALabeledCCC=dfSLA.join(cccFiltered,['nif_cliente'],'inner')
dfSLALabeledCCC.count()

1740807

In [17]:
dfSLALabeledCCC.groupBy('label').count().show()

+-----+-------+
|label|  count|
+-----+-------+
|  0.0|1676067|
|  1.0|  64740|
+-----+-------+



## Churn y factor SLA 


### Ordenes cerradas


In [ ]:
days_range = [30, 60, 90, 120, 180, 240, 365]

#volumen y churn rate

dfLabel=dfSLALabeled.groupBy('label').count()
dfAgr=dfLabel.toPandas()
label0=dfAgr.iloc[0]['count']
label1=dfAgr.iloc[1]['count']  
    
print 'Churn rate:', label1/(label1+label0)*100
print dfLabel.show()

for day in days_range:
    print 'Range Day:',day 
    
    cont=0
        
    var='mean_sla_factor_last'+str(day)
    dfSLALabeledC=dfSLALabeled.filter(col(var)>-1)
    
    while cont <= 2:
    
        if cont==0: 
            dfSLALabeledFilter=dfSLALabeledC.filter(col(var)<1)
            titulo='Max_SLA <1:'
        if cont==1: 
            dfSLALabeledFilter=dfSLALabeledC.filter((col(var)>1) & (col(var)<3))
            titulo='1 < Max_SLA <3:'
        if cont==2: 
            dfSLALabeledFilter=dfSLALabeledC.filter(col(var)>3)
            titulo='Max_SLA > 3:'
        
        dfLabel=dfSLALabeledFilter.groupBy('label').count()
        dfAgr=dfLabel.toPandas()
        label0=dfAgr.iloc[0]['count']
        label1=dfAgr.iloc[1]['count']
        
        print titulo,label1/(label1+label0)*100
        print dfLabel.show()
        cont=cont+1
        
    print '\n'
    

In [ ]:
## Incluyendo el filtro CCC

days_range = [30, 60, 90, 120, 180, 240, 365]

dfLabel=dfSLALabeledCCC.groupBy('label').count()
dfAgr=dfLabel.toPandas()
label0=dfAgr.iloc[0]['count']
label1=dfAgr.iloc[1]['count']  
    
print 'Churn rate:', label1/(label1+label0)*100
print dfLabel.show()

del dfLabel

#volumen y churn rate

for day in days_range:
    print 'Range Day:',day 
    
    cont=0
        
    var='mean_sla_factor_last'+str(day)
        
    while cont <= 2:
    
        if cont==0: 
            dfSLALabeledFilter=dfSLALabeledCCC.filter(col(var)<1)
            titulo='Max_SLA <1:'
        if cont==1: 
            dfSLALabeledFilter=dfSLALabeledCCC.filter((col(var)>1) & (col(var)<3))
            titulo='1 < Max_SLA <3:'
        if cont==2: 
            dfSLALabeledFilter=dfSLALabeledCCC.filter(col(var)>3)
            titulo='Max_SLA > 3:'
        
        dfLabel=dfSLALabeledFilter.groupBy('label').count()
        dfAgr=dfLabel.toPandas()
        label0=dfAgr.iloc[0]['count']
        label1=dfAgr.iloc[1]['count']
    
        print(titulo,label1/(label1+label0)*100)
        print dfLabel.show()
    
        del dfSLALabeledFilter
        del dfLabel
        
        cont=cont+1
        
    print '\n'
    

### Ordenes abiertas


In [18]:
from pyspark.sql.functions import greatest

In [20]:
# Con el total de órdenes abiertas (sin importar el tipo o si están dentro o fuera de SLA)

days_range = [30, 60, 90, 120, 180, 240, 365]

for day in days_range:
    
    print 'Range Day:',day
    selColumns=[]
    for column in dfSLALabeled.columns:
        if 'Max_SLA_factor_running'+str(day) in column:
            selColumns.append(column)

    dfMaxLabeled=(dfSLALabeled.withColumn('Max_'+str(day),greatest(*selColumns)).select(['nif_cliente','label','Max_'+str(day)]))
    #dfMinLabeled=(dfSLALabeled.withColumn('Min_'+str(day),least(*selColumns)).select(['nif_cliente','label','Min_'+str(day)]))

    dfMaxLabeledC=dfMaxLabeled.filter(col('Max_'+str(day))>-1)
    
    dfLabel=dfMaxLabeledC.groupBy('label').agg(count('*'))
    dfAgr=dfLabel.toPandas()
    label0=dfAgr.iloc[0]['count(1)']
    label1=dfAgr.iloc[1]['count(1)']
    
    print 'Churn rate:', label1/(label1+label0)*100
    print dfLabel.show()
        
    var='Max_'+str(day)
    
    cont=0
    
    while cont <= 2:
    
        if cont==0: 
            dfMaxLabeledFilter=dfMaxLabeledC.filter(col(var)<1)
            titulo='Max_SLA <1:'
        if cont==1: 
            dfMaxLabeledFilter=dfMaxLabeledC.filter((col(var)>1) & (col(var)<3))
            titulo='1 < Max_SLA <3:'
        if cont==2: 
            dfMaxLabeledFilter=dfMaxLabeledC.filter(col(var)>3)
            titulo='Max_SLA > 3:'
        
        print(dfMaxLabeledFilter.count())
        dfLabel=dfMaxLabeledFilter.groupBy('label').agg(count('*'))
        dfAgr=dfLabel.toPandas()
        label0=dfAgr.iloc[0]['count(1)']
        label1=dfAgr.iloc[1]['count(1)']
    
        print(titulo,label1/(label1+label0)*100)
        print dfLabel.show()
        
        cont=cont+1
        
    print '\n'


Range Day: 30
Churn rate: 3.556411908087865
+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0| 2211383|
|  1.0|   81546|
+-----+--------+

None
2252419
('Max_SLA <1:', 3.419346045296191)
+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0| 2175401|
|  1.0|   77018|
+-----+--------+

None
8651
('1 < Max_SLA <3:', 12.796208530805686)
+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0|    7544|
|  1.0|    1107|
+-----+--------+

None
28705
('Max_SLA > 3:', 10.88312140742031)
+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0|   25581|
|  1.0|    3124|
+-----+--------+

None


Range Day: 60
Churn rate: 3.556411908087865
+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0| 2211383|
|  1.0|   81546|
+-----+--------+

None
2212395
('Max_SLA <1:', 3.381041812153797)
+-----+--------+
|label|count(1)|
+-----+--------+
|  0.0| 2137593|
|  1.0|   74802|
+-----+--------+

None
10146
('1 < Max_SLA <3:', 11.856889414547606)
+-----+--------+
|label|count(1)|
+-----+--

In [24]:
for column in dfSLALabeled.columns:
    if 'Max_SLA_factor_running30' in column:
        selColumns.append(column)

dfMaxLabeledddd=(dfSLALabeled.withColumn('Max_30',greatest(*selColumns)).select(['nif_cliente','label','Max_30']))
    

In [26]:
dfMaxLabeledddd.filter(col('Max_30')<0).show()

+-----------+-----+------+
|nif_cliente|label|Max_30|
+-----------+-----+------+
+-----------+-----+------+



In [ ]:
# Con el total de órdenes abiertas (sin importar el tipo o si están dentro o fuera de SLA)
## Incluyendo el filtro CCC


days_range = [30, 60, 90, 120, 180, 240, 365]


for day in days_range:
    
    cont=0
    
    print 'Range Day:',day
    selColumns=[]
    for column in dfSLALabeledCCC.columns:
        if 'Max_SLA_factor_running'+str(day) in column:
            selColumns.append(column)
            
    dfMaxLabeled=(dfSLALabeledCCC.withColumn('Max_'+str(day),greatest(*selColumns)).select(['nif_cliente','label','Max_'+str(day)]))
    dfMaxLabeledC=dfMaxLabeled.dropna(subset='Max_'+str(day))
    
    print(dfMaxLabeledC.count())    
    
    dfLabel=dfMaxLabeledC.groupBy('label').agg(count('*'))
    dfAgr=dfLabel.toPandas()
    label0=dfAgr.iloc[0]['count(1)']
    label1=dfAgr.iloc[1]['count(1)'] 
    
    print('Churn Rate',label1/(label1+label0)*100)
    
    var='Max_'+str(day)
    
    while cont <= 2:
    
        if cont==0: 
            dfMaxLabeledFilter=dfMaxLabeledC.filter(col(var)<1)
            titulo='Max_SLA <1:'
        if cont==1: 
            dfMaxLabeledFilter=dfMaxLabeledC.filter((col(var)>1) & (col(var)<3))
            titulo='1 < Max_SLA <3:'
        if cont==2: 
            dfMaxLabeledFilter=dfMaxLabeledC.filter(col(var)>3)
            titulo='Max_SLA > 3:'

        print 'Churn rate:', label1/(label1+label0)*100
        print dfLabel.show()

        print(dfMaxLabeledFilter.count())
        dfLabel=dfMaxLabeledFilter.groupBy('label').agg(count('*'))
        dfAgr=dfLabel.toPandas()
        label0=dfAgr.iloc[0]['count(1)']
        label1=dfAgr.iloc[1]['count(1)']

        print(titulo,label1/(label1+label0)*100)

        del dfMaxLabeledFilter
        del dfLabel

        cont=cont+1
    
   

In [ ]:
selColumns=[]
day=30
for column in dfSLALabeled.columns:
    if 'Max_SLA_factor_running'+str(day) in column:
        selColumns.append(column)

In [ ]:
selColumns

In [ ]:
dfMaxx=(dfSLALabeled.withColumn('Max_30',greatest(*selColumns)).select(['nif_cliente','label','Max_30']))
dfMaxx.show()

In [ ]:
dfMaxx.filter((col('Max_30')<3) & (col('Max_30')>1)).count()

In [ ]:
dfMaxxx=(dfSLA.withColumn('Max_30',greatest(*selColumns)).select(['nif_cliente','Max_30']))

dfMaxxx.filter((col('Max_30')<3) & (col('Max_30')>1)).count()

In [ ]:
disminucion_Max_SLA_factor_running30_OutsideSLA
disminucion_Max_SLA_factor_running60_OutsideSLA
disminucion_Max_SLA_factor_running90_OutsideSLA
disminucion_Max_SLA_factor_running120_OutsideSLA
disminucion_Max_SLA_factor_running180_OutsideSLA
disminucion_Max_SLA_factor_running240_OutsideSLA
disminucion_Max_SLA_factor_running365_OutsideSLA
disminucion_Max_SLA_factor_running30_InsideSLA
disminucion_Max_SLA_factor_running60_InsideSLA
disminucion_Max_SLA_factor_running90_InsideSLA
disminucion_Max_SLA_factor_running120_InsideSLA
disminucion_Max_SLA_factor_running180_InsideSLA
disminucion_Max_SLA_factor_running240_InsideSLA
disminucion_Max_SLA_factor_running365_InsideSLA

In [ ]:
days_range = [30, 60, 90, 120, 180, 240, 365]

for day in days_range:
    print(day)
    print('Label1/(Label0+Label1) %')
    
    var='mean_sla_factor_last'+str(day)
        
    dfSLALabeledFilter=dfSLALabeled.filter(col(var)<1)
    dfLabel=dfSLALabeledFilter.groupBy('label').agg(count('*'))
    dfAgr=dfLabel.toPandas()
    label0=dfAgr.iloc[0]['count(1)']
    label1=dfAgr.iloc[1]['count(1)']
    
    print('Filtro <1:',label1/(label1+label0)*100)
        
    dfSLALabeledFilter=dfSLALabeled.filter((col(var)>1) & (col(var)<3))
    dfLabel=dfSLALabeledFilter.groupBy('label').agg(count('*'))
    dfAgr=dfLabel.toPandas()
    label0=dfAgr.iloc[0]['count(1)']
    label1=dfAgr.iloc[1]['count(1)']
    
    print('Filtro >1 y <3:',label1/(label1+label0)*100)
        
    dfSLALabeledFilter=dfSLALabeled.filter(col(var)>3)
    dfLabel=dfSLALabeledFilter.groupBy('label').agg(count('*'))
    dfAgr=dfLabel.toPandas()
    label0=dfAgr.iloc[0]['count(1)']
    label1=dfAgr.iloc[1]['count(1)']
    
    print('Filtro >3:',label1/(label1+label0)*100)

In [ ]:
churn factor_SLA

In [ ]:
mas churn mas se va de SLA

In [ ]:
churn tipos

In [ ]:
customer_master.groupBy('label').count().show()